In [ ]:
import psycopg2
import undetected_chromedriver as uc
uc.install()
from selectorlib import Extractor
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import Chrome
from fake_useragent import UserAgent
import time
import random

import pandas as pd
import numpy as np

In [ ]:
birthdays = [ '12/14/56', '06/09/67','09/13/66', '02/14/60', '05/01/40', '03/14/91', '01/09/44',
             '11/12/69', '06/09/86', '09/25/95', '04/20/72', '03/08/59', '08/20/90', '01/15/48', 
             '10/16/44', '10/30/55', '06/25/59', '06/03/58', '04/21/62', '11/29/54', '07/20/70',
             '12/18/71', '06/04/89', '06/02/40', '10/16/80', '08/23/74', '09/24/43', '05/17/80',
             '02/07/51', '01/22/44', '06/09/99', '02/23/97', '05/06/64', '02/17/68', '07/11/52',
             '05/10/61', '06/10/60', '02/25/41', '06/07/02', '02/08/89', '01/27/95', '02/13/73',
             '09/12/80', '07/18/88', '02/22/64', '11/26/54', '06/28/50', '03/20/85', '09/03/59'
    
]

In [ ]:
# Establish a connection to the database by creating a cursor object
# The PostgreSQL server must be accessed through the PostgreSQL APP or Terminal Shell

# conn = psycopg2.connect("dbname=suppliers port=5432 user=postgres password=postgres")

# Or:
conn = psycopg2.connect(host="localhost", port = 5432, database="addresses", user="postgres", password="password")

# Create a cursor object
cur = conn.cursor()

# A sample query of all data from the "vendors" table in the "suppliers" database
cur.execute("""SELECT *
FROM residents
FULL OUTER JOIN house
ON residents.address_id = house.address_id
WHERE post_comm ILIKE('hackettstown');""")
query_results = cur.fetchall()

# Close the cursor and connection to so the server can allocate
# bandwidth to other requests
cur.close()
conn.close()

In [ ]:
query_results

In [ ]:
grab_vars = query_results[1]
grab_vars
address_id = grab_vars[1]
name = grab_vars[12]
zipcode = grab_vars[20]
streetname = grab_vars[21]
streetname

In [ ]:
position = 0
for i, house in enumerate(query_results):
    position += 1
    
    capa = DesiredCapabilities.CHROME
    capa["pageLoadStrategy"] = 'none'
    
    
    PROXY = '198.204.241.50:17057'
    
    
    #### Start browser
    #ChromeOptions = webdriver.ChromeOptions()
    ChromeOptions = uc.ChromeOptions()
    ua = UserAgent()
    userAgent = ua.safari
    print(userAgent)
    ChromeOptions.add_argument('--incognito')
    ChromeOptions.add_argument(f'user-agent={userAgent}')
    #ChromeOptions.add_argument('--headless')
    ChromeOptions.add_argument('--proxy-server=198.204.241.50:17057')

    capa['proxy'] = {
            "httpProxy": PROXY,
            "ftpProxy": PROXY,
            "sslProxy": PROXY,
            "proxyType": "MANUAL",

    }
    
    capa['acceptSslCerts'] = True
    
    
    
    #executable_path=ChromeDriverManager().install()
    #driver = webdriver.Chrome(options=ChromeOptions, desired_capabilities=capa)
    driver = Chrome(options=ChromeOptions, desired_capabilities=capa)
    driver.delete_all_cookies()

    
    time.sleep(random.uniform(2,3.75))
    

    
    
    #establish database connection
    conn = psycopg2.connect(host="localhost", port = 5432, database="addresses", user="postgres", password="password")
    address_id = house[1]
    name = house[12]
    zipcode = house[20]
    streetname = house[21]
    dob = random.choice(birthdays)
    
    timeout = random.uniform(73,88)
    wait = WebDriverWait(driver, timeout)
    
    
    try:
        driver.get("https://www.progressive.com")
        wait.until(EC.presence_of_element_located((By.XPATH, '//span[contains(text(), "Auto") and @class="last-word"]')))

        time.sleep(random.uniform(.1,2.3))
        driver.execute_script("window.stop();")
        print('element loaded')
        try:
            car_quote = driver.find_element_by_xpath('//span[contains(text(), "Auto") and @class="last-word"]')
            time.sleep(random.uniform(.2,2))
            car_quote.click()
        except:
            car_quote = driver.find_element_by_xpath('//p[contains(text(), "Auto ") and @class="txt"]')
            time.sleep(random.uniform(.2,2))
            car_quote.click()


    except TimeoutException:
        print("Timed out waiting for page to load!")
    except WebDriverException as e:
        print('SOMETHING WENT WRONG', e)
        pass
    except Exception as e:
        print('Error: !!!!!: ', e)
        pass
    finally:
        print("Address: ", streetname)
        
    
    time.sleep(random.uniform(2,4))
    
    zipcode_entry = driver.find_element_by_css_selector('#zipCode_overlay')
    
    zipcode_entry.send_keys(zipcode)
    
    zipcode_entry.send_keys(Keys.RETURN)
    
    time.sleep(2)
    
    get_quote = driver.find_element_by_xpath('//*[@id="QuoteStartForm_overlay"]/div[5]')
    get_quote.click()
    
    time.sleep(10)
    
    print('PASS')
    
    
    
    raw_html = driver.page_source
    
#     resident_info = Extractor.from_yaml_string("""
#     main_card:
#         css: 'div.ThatsThem-record:nth-of-type(n+2) div.ThatsThem-well'
#         xpath: null
#         multiple: true
#         type: Text
#         children:
#             address:
#                 css: div.ThatsThem-record-address
#                 xpath: null
#                 multiple: true
#                 type: Text
#             phone_number:
#                 css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(1)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             alt_phone:
#                 css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(2)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             email:
#                 css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(3)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             length_of_residence:
#                 css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(4)'
#                 xpath: null
#                 type: Text
#             household_size:
#                 css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(5)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             ip_address:
#                 css: 'dd.col-md-8:nth-of-type(6)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             net_worth:
#                 css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(1)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             estimated_income:
#                 css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(2)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             education:
#                 css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(3)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             occupation:
#                 css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(4)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             language:
#                 css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(5)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             name:
#                 css: 'h2 span'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             """)
#     raw_data = resident_info.extract(raw_html)
    
#     df = pd.DataFrame(raw_data['main_card'])
    
#     df = df.apply(unlist)
    
#     df['address_id'] = house[-1]
    
#     for row in list(range(len(df))):
#         data = df.iloc[row].values

#         address_id = str(data[-1])
#         resident_phone = str(data[1])
#         alt_phones = str(data[2])
#         email = str(data[3])
#         resident_length = str(data[4])
#         household_size = str(data[5])
#         ip_address = str(data[6])
#         net_worth = str(data[7])
#         estimated_income = str(data[8])
#         education = str(data[9])
#         occupation = str(data[10])
#         name = str(data[12])
#         print("saving: ", name)

#         cur = conn.cursor()

#         cur.execute("""INSERT INTO residents(address_id, resident_phone, alt_phones, email, resident_length, household_size, ip_address, net_worth, estimated_income, education, occupation, name) VALUES
#         (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""",(address_id, resident_phone, alt_phones, email, resident_length, household_size, ip_address, net_worth, estimated_income, education, occupation, name))
#         conn.commit()

#         cur.close()
#     conn.close()
#     time.sleep(random.uniform(.1,.3))
    driver.close()

In [ ]:
car_quote = driver.find_element_by_xpath('//span[contains(text(), "Auto") and @class="last-word"]')

In [ ]:
house = grab_vars
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = 'none'


PROXY = '198.204.241.50:17057'


#### Start browser
#ChromeOptions = webdriver.ChromeOptions()
ChromeOptions = uc.ChromeOptions()
ua = UserAgent()
userAgent = ua.safari
print(userAgent)
ChromeOptions.add_argument('--incognito')
ChromeOptions.add_argument(f'user-agent={userAgent}')
#ChromeOptions.add_argument('--headless')
ChromeOptions.add_argument('--proxy-server=198.204.241.50:17057')

capa['proxy'] = {
        "httpProxy": PROXY,
        "ftpProxy": PROXY,
        "sslProxy": PROXY,
        "proxyType": "MANUAL",

}

capa['acceptSslCerts'] = True



#executable_path=ChromeDriverManager().install()
#driver = webdriver.Chrome(options=ChromeOptions, desired_capabilities=capa)
driver = Chrome(options=ChromeOptions, desired_capabilities=capa)
driver.delete_all_cookies()


time.sleep(random.uniform(2,3.75))




#establish database connection
conn = psycopg2.connect(host="localhost", port = 5432, database="addresses", user="postgres", password="password")
address_id = house[1]
name = house[12]
zipcode = house[20]
streetname = house[21]
dob = random.choice(birthdays)

timeout = random.uniform(73,88)
wait = WebDriverWait(driver, timeout)


try:
    driver.get("https://www.progressive.com")
    wait.until(EC.presence_of_element_located((By.XPATH, '//span[contains(text(), "Auto") and @class="last-word"]')))

    time.sleep(random.uniform(.1,2.3))
    driver.execute_script("window.stop();")
    print('element loaded')
    try:
        car_quote = driver.find_element_by_xpath('//span[contains(text(), "Auto") and @class="last-word"]')
        time.sleep(random.uniform(.2,2))
        car_quote.click()
    except:
        car_quote = driver.find_element_by_xpath('//p[contains(text(), "Auto ") and @class="txt"]')
        time.sleep(random.uniform(.2,2))
        car_quote.click()


except TimeoutException:
    print("Timed out waiting for page to load!")
except WebDriverException as e:
    print('SOMETHING WENT WRONG', e)
    pass
except Exception as e:
    print('Error: !!!!!: ', e)
    pass
finally:
    print("Address: ", streetname)


time.sleep(random.uniform(2,4))

zipcode_entry = driver.find_element_by_css_selector('#zipCode_overlay')

zipcode_entry.send_keys(zipcode)

zipcode_entry.send_keys(Keys.RETURN)

# time.sleep(2)

# get_quote = driver.find_element_by_xpath('//*[@id="QuoteStartForm_overlay"]/div[5]')
# get_quote.click()

time.sleep(10)

print('PASS')

In [ ]:
car_quote = driver.find_element_by_css_selector('#main > div.mma.orange > div > div > div.products-wrap > div.products-bundle > ul > li:nth-child(1) > a > span > img')
time.sleep(random.uniform(.2,2))
car_quote.click()

In [ ]:
#main > div.mma.orange > div > div > div.products-wrap > div.products-bundle > ul > li:nth-child(1) > a > p

In [ ]:
#main > div.mma.orange > div > div > div.products-wrap > div.products-bundle > ul > li:nth-child(1) > a > p

In [ ]:
# signal TOR for a new connection
def switchIP():
    with Controller.from_port(port = 9051) as controller:
        controller.authenticate()
        controller.signal(Signal.NEWNYM)

# get a new selenium webdriver with tor as the proxy
def my_proxy(PROXY_HOST,PROXY_PORT):
    fp = webdriver.FirefoxProfile()
    # Direct = 0, Manual = 1, PAC = 2, AUTODETECT = 4, SYSTEM = 5
    fp.set_preference("network.proxy.type", 1)
    fp.set_preference("network.proxy.socks",PROXY_HOST)
    fp.set_preference("network.proxy.socks_port",int(PROXY_PORT))
    fp.update_preferences()
    options = Options()
    #options.headless = True
    return webdriver.Firefox(options=options, firefox_profile=fp)
    
    
for i, car in enumerate(query_results[:1722]):
    car_pos = i
    

    
    
    #establish database connection
#     conn = psycopg2.connect(host="localhost", port = 5432, database="addresses", user="postgres", password="password")
#     town = car[1]
#     street = car[8]
#     zipcode = car[7]
#     num =car[4]
#     st = car[2]
#     state = car[0]
#     address_to_input = street + ' ' + ' ' + town + ' ' + state + ' ' + zipcode
#     address_to_input = address_to_input.replace('  ', ' ').lower()
#     address_to_input = address_to_input.replace(' ', '-').lower()
#     time.sleep(1)
    #direct link
    
    proxy = my_proxy("127.0.0.1", 9050)
    
    proxy.get('https://www.geico.com/')
    
    proxy.find_element_by_xpath('/html/body/div[2]/div/div/div[1]/div/section/div[1]/div/div/form/div[2]/div[1]/div[2]').click()
    
    
    proxy.find_element_by_id('zip').send_keys('TEST')
    
    time.sleep(2)
    raw_html = proxy.page_source
    
    
    
    
#     resident_info = Extractor.from_yaml_string("""
#     main_card:
#         css: 'div.ThatsThem-record:nth-of-type(n+2) div.ThatsThem-well'
#         xpath: null
#         multiple: true
#         type: Text
#         children:
#             address:
#                 css: div.ThatsThem-record-address
#                 xpath: null
#                 multiple: true
#                 type: Text
#             phone_number:
#                 css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(1)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             alt_phone:
#                 css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(2)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             email:
#                 css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(3)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             length_of_residence:
#                 css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(4)'
#                 xpath: null
#                 type: Text
#             household_size:
#                 css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(5)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             ip_address:
#                 css: 'dd.col-md-8:nth-of-type(6)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             net_worth:
#                 css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(1)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             estimated_income:
#                 css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(2)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             education:
#                 css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(3)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             occupation:
#                 css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(4)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             language:
#                 css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(5)'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             name:
#                 css: 'h2 span'
#                 xpath: null
#                 multiple: true
#                 type: Text
#             """)
#     raw_data = resident_info.extract(raw_html)
    
#     df = pd.DataFrame(raw_data['main_card'])
    
#     df = df.apply(unlist)
    
#     df['address_id'] = house[-1]
    
#     for row in list(range(len(df))):
#         data = df.iloc[row].values

#         address_id = str(data[-1])
#         resident_phone = str(data[1])
#         alt_phones = str(data[2])
#         email = str(data[3])
#         resident_length = str(data[4])
#         household_size = str(data[5])
#         ip_address = str(data[6])
#         net_worth = str(data[7])
#         estimated_income = str(data[8])
#         education = str(data[9])
#         occupation = str(data[10])
#         name = str(data[12])
#         print("saving: ", name)

#         cur = conn.cursor()

#         cur.execute("""INSERT INTO residents(address_id, resident_phone, alt_phones, email, resident_length, household_size, ip_address, net_worth, estimated_income, education, occupation, name) VALUES
#         (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""",(address_id, resident_phone, alt_phones, email, resident_length, household_size, ip_address, net_worth, estimated_income, education, occupation, name))
#         conn.commit()

#         cur.close()
#     conn.close()
#     time.sleep(.5)
    proxy.close()
    switchIP()